This notebook is looking at metadata from SRA. I am wanting to develop a programmatic approach to download metadata information, but needs to be consistent with Zhenxia and Miegs analysis. I am also wanting to figure exactly how Zhenxia and the Miegs did this.

There are two programmatic approaches:
* SRAdb
* Biopython

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 999)

# SRAdb

SRAdb is nice because it parses everything into a table, but the counts don't match with my expectations. 

In [2]:
import rpy2
%load_ext rpy2.ipython

## Download SQLite database

In [ ]:
%%bash
cd ../../output/
if [ ! -e SRAmetadb.sqlite ]; then 
    wget 'http://dl.dropbox.com/u/51653511/SRAmetadb.sqlite.gz'
    gunzip SRAmetadb.sqlite.gz
fi

## Query database

In [4]:
%%R 
library(SRAdb)
sra_con = dbConnect(SQLite(), '../../output/SRAmetadb.sqlite')
rs = getSRA(search_terms='"Drosophila melanogaster"[Organism]', out_types=c('run', 'sample'), sra_con=sra_con)

## Print number of rows

In [7]:
%R dim(rs)

array([10356,    21], dtype=int32)

# Biopyton

Biopython's counts match what I expect, but the results are in raw XML and need a parser. 

In [4]:
from Bio import Entrez
Entrez.email = 'justin.fear@nih.gov'

## Get List of Records

In [5]:
handle = Entrez.esearch(db='sra', term='"Drosophila melanogaster"[Orgn]')
records = Entrez.read(handle)
records['Count']

'22344'

## Get Full XML of Subset

In [11]:
h = Entrez.efetch(db='sra', id=records['IdList'][:1])
r = h.read()

print(r)

<?xml version="1.0" ?>
<EXPERIMENT_PACKAGE_SET>
<EXPERIMENT_PACKAGE><EXPERIMENT alias="E-MTAB-1976:C1" accession="ERX330955" broker_name="ArrayExpress"><IDENTIFIERS><PRIMARY_ID>ERX330955</PRIMARY_ID><SUBMITTER_ID namespace="UNIVERSITY OF MANCHESTER">E-MTAB-1976:C1</SUBMITTER_ID><SUBMITTER_ID namespace="University of Manchester">E-MTAB-1976:C1</SUBMITTER_ID></IDENTIFIERS><TITLE>AB SOLiD 4 System sequencing; Deignan</TITLE><STUDY_REF refname="E-MTAB-1976" refcenter="University of Manchester" accession="ERP004143"><IDENTIFIERS><PRIMARY_ID>ERP004143</PRIMARY_ID><SUBMITTER_ID namespace="University of Manchester">E-MTAB-1976</SUBMITTER_ID></IDENTIFIERS></STUDY_REF><DESIGN><DESIGN_DESCRIPTION>Deignan</DESIGN_DESCRIPTION><SAMPLE_DESCRIPTOR refname="E-MTAB-1976:C1" accession="ERS362580" refcenter="University of Manchester"><IDENTIFIERS><PRIMARY_ID>ERS362580</PRIMARY_ID><EXTERNAL_ID namespace="BioSample">SAMEA2225913</EXTERNAL_ID><SUBMITTER_ID namespace="University of Manchester">E-MTAB-1976:C1<

# SRA Website

The gold standard is downloading results directly from SRA's website. This is what Zhenxia and the Miegs did. I still want to verify how they made their tables.

In [6]:
# see ../../data/SRA/README for info about where this file came from
dfSRA = pd.read_csv('../../data/SRA/SraRunTable.txt', sep='\t')
dfSRA.set_index('Run_s', inplace=True)
print(dfSRA.shape)
dfSRA.head()


(13993, 29)


,Assay_Type_s,AssemblyName_s,BioProject_s,BioSample_s,Center_Name_s,Experiment_s,InsertSize_l,LibraryLayout_s,LibrarySelection_s,LibrarySource_s,Library_Name_s,LoadDate_s,MBases_l,MBytes_l,Platform_s,ReleaseDate_s,SRA_Sample_s,SRA_Study_s,Sample_Name_s,Sex_s,genotype_s,source_name_s,strain_s,tissue_s,Consent_s,Organism_s,g1k_analysis_group_s,g1k_pop_code_s,source_s
Run_s,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
SRR000909,FL-cDNA,<not provided>,PRJNA74619,SAMN00000114,WUGSC,SRX000185,0,SINGLE,cDNA,TRANSCRIPTOMIC,2305754913,2012-01-19,0,0,LS454,2008-04-09,SRS000346,SRP000119,SRA000293_sample,<not provided>,<not provided>,<not provided>,<not provided>,<not provided>,public,Drosophila melanogaster,<not provided>,<not provided>,<not provided>
SRR001337,EST,<not provided>,PRJNA107113,SAMN00000196,GEO,SRX000316,0,SINGLE,RANDOM,TRANSCRIPTOMIC,WT_females_beta-eliminated,2014-05-25,54,362,ILLUMINA,2008-06-27,SRS000430,SRP000181,GSM278573,<not provided>,<not provided>,<not provided>,<not provided>,<not provided>,public,Drosophila melanogaster,<not provided>,<not provided>,<not provided>
SRR001338,EST,<not provided>,PRJNA107113,SAMN00000208,GEO,SRX000328,0,SINGLE,RANDOM,TRANSCRIPTOMIC,IR_non-beta-eliminated,2014-05-25,115,796,ILLUMINA,2008-06-27,SRS000442,SRP000181,GSM278707,<not provided>,<not provided>,<not provided>,<not provided>,<not provided>,public,Drosophila melanogaster,<not provided>,<not provided>,<not provided>
SRR001339,EST,<not provided>,PRJNA107113,SAMN00000207,GEO,SRX000327,0,SINGLE,RANDOM,TRANSCRIPTOMIC,WT_females_non-beta-eliminated,2014-05-25,144,1008,ILLUMINA,2008-06-27,SRS000441,SRP000181,GSM278706,<not provided>,<not provided>,<not provided>,<not provided>,<not provided>,public,Drosophila melanogaster,<not provided>,<not provided>,<not provided>
SRR001340,EST,<not provided>,PRJNA107113,SAMN00000198,GEO,SRX000318,0,SINGLE,RANDOM,TRANSCRIPTOMIC,IR_beta-eliminated,2014-05-25,63,425,ILLUMINA,2008-06-27,SRS000432,SRP000181,GSM278575,<not provided>,<not provided>,<not provided>,<not provided>,<not provided>,public,Drosophila melanogaster,<not provided>,<not provided>,<not provided>


In [14]:
dfSRA.columns

Index(['Assay_Type_s', 'AssemblyName_s', 'BioProject_s', 'BioSample_s',
       'Center_Name_s', 'Experiment_s', 'InsertSize_l', 'LibraryLayout_s',
       'LibrarySelection_s', 'LibrarySource_s', 'Library_Name_s', 'LoadDate_s',
       'MBases_l', 'MBytes_l', 'Platform_s', 'ReleaseDate_s', 'Run_s',
       'SRA_Sample_s', 'SRA_Study_s', 'Sample_Name_s', 'Sex_s', 'genotype_s',
       'source_name_s', 'strain_s', 'tissue_s', 'Consent_s', 'Organism_s',
       'g1k_analysis_group_s', 'g1k_pop_code_s', 'source_s'],
      dtype='object')

In [19]:
# List of tissue types, note that there is some cleaning that needs done
dfSRA.tissue_s.unique()

array(['<not provided>', 'whole organism', 'heads', 'head', 'testis',
       'ovary', 'whole fly', 'testes', 'ovaries', 'mixed-stage embryos',
       'female heads', 'embryo-derived cell-line', 'Adult ovaries',
       'Male body', 'CNS-derived cell-line', 'dorsal mesothoracic disc',
       'antenna disc-derived cell-line', 'ventral prothoracic disc',
       'Female heads', 'fat body', 'midgut', 'Ovary', 'Whole flies',
       'brain', 'whole flies', 'fly', 'Immortalized cells',
       'Adult fly heads', 'salivary gland', 'male whole body',
       'female whole body', 'central nervous system', 'whole brains',
       'mushroom bodies', 'Head', 'Digestive System', 'Accessory Gland',
       'Carcass', 'Imaginal Disc', 'Central Nervous System', 'Fat',
       'Ovaries', 'Testes', 'Salivary Gland', 'neuron', 'muscle',
       'whole-embryo', 'eggs', 'whole body without testis', 'whole body',
       'Brain', 'dissected testis', 'ovary stem cells',
       'embryonic/larval hemocyte', 'Heads', 'Wi

# Zhen-Xia's List

In [20]:
# Zhenxia gave me this dataset, it is names `.xls` but it is really a `tsv`.
dfZ = pd.read_table('../../data/zhenxia/metadata_quality_control.xls')
dfZ.set_index('SRR', inplace=True)
print(dfZ.shape)
dfZ.head()

(12719, 57)


,BioSample,ReleaseDate,LoadDate,avgLength,size_MB,Experiment,LibraryName,LibraryStrategy,LibrarySelection,LibrarySource,LibraryLayout,Platform,Model,SRAStudy,BioProject,Sample,SampleName,CenterName,Submission,sex,dev_stage,tissue,genotype,cell_type,ExperimentTitle,StudyTitle,sample_type,Total_reads,Unmapped_reads,non.unique,unique,Non.splice_reads,Splice_reads,ambiguous,no_feature,reads_mapped2genes,reads_mapped2intergenic,reads_mapped2ERCC,mRNA,ncRNA,pseudogene,rRNA,snoRNA,snRNA,tRNA,exp_genes,layout,failed,F,R,TIN_median,median_coverage_1.20,median_coverage_21.40,median_coverage_41.60,median_coverage_61.80,median_coverage_81.100,correlation
SRR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
DRR001177,SAMD00016337,2011-12-11,2014-05-31,36,832,DRX000774,SRig10098,OTHER,size fractionation,OTHER,SINGLE,ILLUMINA,Illumina Genome Analyzer IIx,DRP000426,PRJDB2780,DRS000734,SAMD00016337,RIKEN_OSC,DRA000418,male,embryo,whole_organism,AGO2- and negative control-IP short RNA-seq in...,S2,SRig10098,Chromatin associated RNAi components take part...,cell_line,38551396,24102998,11711140,2737258,2737191,67,63300,542516,1687592,443596,254,432466,1182063,54793,5219,9665,165,3221,13691,SINGLE,0.1378,0.4379,0.4243,6.250000,0.080266,0.079628,0.216993,0.341371,0.604689,0.693731
DRR001178,SAMD00016336,2011-12-11,2014-05-31,36,791,DRX000775,SRig10099,OTHER,size fractionation,OTHER,SINGLE,ILLUMINA,Illumina Genome Analyzer IIx,DRP000426,PRJDB2780,DRS000735,SAMD00016336,RIKEN_OSC,DRA000418,male,embryo,whole_organism,AGO2- and negative control-IP short RNA-seq in...,S2,SRig10099,Chromatin associated RNAi components take part...,cell_line,36613554,24010803,9923336,2679415,2679391,24,98459,515561,1643139,422006,250,657744,889254,56460,4831,30900,509,3441,13634,SINGLE,0.1688,0.4294,0.4018,7.945317,0.064579,0.081030,0.143947,0.199425,0.726678,0.693217
DRR014222,SAMD00012584,2015-03-23,2015-03-23,101,10621,DRX012753,PCR2085,EST,cDNA,TRANSCRIPTOMIC,SINGLE,ILLUMINA,Illumina HiSeq 2500,DRP001250,PRJDB2557,DRS012553,DRS012553,ASGRC-WM,DRA001188,unknown,adult,whole_organism,Clontech Catalog number: 636222 Lot number: 10...,mix,SMART method,mRNA 5 end sequencing,whole_organism,164160619,66074703,3403887,94682029,73561370,21120659,8827565,279985,85422641,151838,0,64520581,3803807,14974,17058229,9806,13724,1520,15046,SINGLE,0.0103,0.9856,0.0041,33.202892,0.513654,0.034357,0.005381,0.068175,0.212674,0.798120
DRR014223,SAMD00012584,2015-03-23,2015-03-23,101,10314,DRX012754,PCR2072,EST,cDNA,TRANSCRIPTOMIC,SINGLE,ILLUMINA,Illumina HiSeq 2500,DRP001250,PRJDB2557,DRS012553,DRS012553,ASGRC-WM,DRA001188,unknown,adult,whole_organism,Clontech Catalog number: 636222 Lot number: 10...,mix,Ligation method,mRNA 5 end sequencing,whole_organism,151593644,31527578,10224830,109841236,38175966,71665270,6497407,181333,103064996,97500,0,102778950,252968,5790,15696,1402,10024,166,14014,SINGLE,0.1035,0.8922,0.0043,17.325735,0.663610,0.096432,0.011929,0.004769,0.032353,0.657597
DRR014224,SAMD00012584,2015-03-23,2015-03-23,126,12218,DRX012755,PCR2123,EST,cDNA,TRANSCRIPTOMIC,SINGLE,ILLUMINA,Illumina HiSeq 2500,DRP001250,PRJDB2557,DRS012553,DRS012553,ASGRC-WM,DRA001188,unknown,adult,whole_organism,Clontech Catalog number: 636222 Lot number: 10...,mix,CapSMART and Non-CapSMART method,mRNA 5 end sequencing,whole_organism,150847202,63307153,3283707,84256342,61657329,22599013,7208452,163524,76794434,89932,0,50509969,9533770,8403,16719380,6275,16301,336,14664,SINGLE,0.0066,0.9926,0.0009,34.692736,0.645220,0.088640,0.004934,0.095900,0.309740,0.785452


# Jean and Danielle's

In [23]:
# I got this table from Jean and Danielle
dfj = pd.read_table('../../data/jean/Mapping_QC/dd_any.Data_Summary.runs.txt', skiprows=2, low_memory=False, index_col=0)
print(dfj.shape)
dfj.head()

(9368, 291)


,Unnamed: 1,Genes significantly expressed,% fragments on strand plus of genes,Unnamed: 4,RunId,Run accession in SRA,Title,Other title,Sorting_title 1,Sorting title 2,Sample (summarized from biosample or manual),Stage and experimental summary,System or Tissue,"Laboratory, Author when available, submission date",Species,Project,Project description,Reference,Unnamed: 19,Sequencing protocol,Design,Sequencing platform and Machine type,Experiment type,Sequencing date,Submission date,Library preparation,RNA Integrity Number,Details,Corresponding microarray,"Spots, bases, average read length (in SRA)",Single or Paired end,Unnamed: 32,Average read length (nt),Fragment multiplicity,Million distinct sequences,Million raw reads,Megabases sequenced,%A,%T,%G,%C,%N,%GC,Unnamed: 44,Exit adaptor of read 1,Exit adaptor of read 2,PolyA index : fragments per million with at least 3 A13 motifs shifted by at least 6 bases,Telomere index : fragments per million with at least 3 TTAGGG/CCCTAA motifs,Telomere index allowing G/C substitution,Noise in telomere index : FPM with at least 3 AATCCC/GGGATT motifs,Unnamed: 51,Million reads aligned on all targets per CPU hour,Maximum RAM used (GB),Number of blocks,Unnamed: 55,Million reads aligned on any target,Average length aligned per read (nt),Mb aligned on any target,%Reads aligned on any target,%Mb aligned on any target,% length aligned on average,Unnamed: 62,Raw reads,Well mapped reads,Reads mapping uniquely to a single locus (genomic site) and maximum 1 gene,"Reads mapping uniquely to a single locus, but to 2 genes in antisense",Reads mapping uniquely to a single locus,Reads mapping to 2 to 9 sites,Reads rejected because they map to 10 sites or more,Reads rejected by alignment quality filter,Reads rejected before alignment because of low entropy (<= 16 bp),Reads with no insert (adaptor only),Reads unaligned from half aligned fragments,"Reads of good quality from unaligned fragments (missing genome, microbiome or mapping difficulty)",% well mapped reads,% reads mapping uniquely to a single locus (genomic site) and maximum 1 gene,"% reads mapping uniquely to a single locus, but to 2 genes in antisense",% reads mapping uniquely to a single locus,% reads mapping to 2 to 9 sites,% reads rejected because they map to 10 sites or more,% reads rejected by alignment quality filter,% reads rejected before alignment because of low entropy (<= 16 bp),% reads with no insert (adaptor only),% reads unaligned from half aligned fragments,"% reads of good quality from unaligned fragments (missing genome, microbiome or mapping difficulty)",Unnamed: 86,Reads aligned in PhiX DNA spike-in,Reads aligned in ERCC RNA spike-in,Reads aligned in ribosomal RNA,Reads aligned in mitochondria,Reads aligned in other small genes,Reads aligned in AceView,Reads aligned in RefSeq,Reads aligned in Encode,Reads aligned in Magic,Reads aligned in genome,Reads aligned in any previously annotated transcripts,"Reads inside introns, new exons and intergenic",Reads aligned in microbiome,Reads aligned in viruses,Reads aligned on the imaginary genome (mapping specificity control),Reads supporting known exons-junctions,Unnamed: 103,% Reads aligned in PhiX DNA spike-in,% Reads aligned in ERCC RNA spike-in,% Reads aligned in ribosomal RNA,% Reads aligned in mitochondria,% Reads aligned in other small genes,% Reads aligned in AceView,% Reads aligned in RefSeq,% Reads aligned in Encode,% Reads aligned in Magic,% Reads aligned in genome,% Reads aligned in any previously annotated transcripts,"% Reads inside introns, new exons and intergenic",% Reads aligned in microbiome,% Reads aligned in viruses,% Reads aligned on the imaginary genome (mapping specificity control),% Reads supporting known exons-junctions,Unnamed: 120,Mb aligned in PhiX DNA spike-in,Mb aligned in ERCC RNA spike-in,Mb aligned in ribosomal RNA,Mb aligned in mitochondria,Mb aligned in other small genes,Mb aligned in AceView,Mb aligned in RefSeq,Mb aligned in Encode,Mb aligned in Magic,Mb alig

In [26]:
for c in dfj.columns:
    print(c)

Unnamed: 1
Genes significantly expressed
% fragments on strand plus of genes
Unnamed: 4
RunId 
Run accession in SRA
Title
Other title
Sorting_title 1
Sorting title 2
Sample (summarized from biosample or manual)
Stage and experimental summary
System or Tissue
Laboratory, Author when available, submission date
Species
Project
Project description
Reference
Unnamed: 19
Sequencing protocol
Design
Sequencing platform and Machine type
Experiment type
Sequencing date
Submission date
Library preparation
RNA Integrity Number
Details
Corresponding microarray
Spots, bases, average read length (in SRA)
Single or Paired end
Unnamed: 32
Average read length (nt)
Fragment multiplicity
Million distinct sequences
Million raw reads
Megabases sequenced
%A
%T
%G
%C
%N
%GC
Unnamed: 44
Exit adaptor of read 1
Exit adaptor of read 2
PolyA index : fragments per million with at least 3 A13 motifs shifted by at least 6 bases
Telomere index : fragments per million with at least 3 TTAGGG/CCCTAA motifs
Telomere index

# Compare Z vs J

In [27]:
zList = dfZ.BioSample.unique()

In [35]:
print(Entrez.einfo().read())

<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE eInfoResult PUBLIC "-//NLM//DTD einfo 20130322//EN" "http://eutils.ncbi.nlm.nih.gov/eutils/dtd/20130322/einfo.dtd">
<eInfoResult>
<DbList>

	<DbName>pubmed</DbName>
	<DbName>protein</DbName>
	<DbName>nuccore</DbName>
	<DbName>nucleotide</DbName>
	<DbName>nucgss</DbName>
	<DbName>nucest</DbName>
	<DbName>structure</DbName>
	<DbName>genome</DbName>
	<DbName>annotinfo</DbName>
	<DbName>assembly</DbName>
	<DbName>bioproject</DbName>
	<DbName>biosample</DbName>
	<DbName>blastdbinfo</DbName>
	<DbName>books</DbName>
	<DbName>cdd</DbName>
	<DbName>clinvar</DbName>
	<DbName>clone</DbName>
	<DbName>gap</DbName>
	<DbName>gapplus</DbName>
	<DbName>grasp</DbName>
	<DbName>dbvar</DbName>
	<DbName>gene</DbName>
	<DbName>gds</DbName>
	<DbName>geoprofiles</DbName>
	<DbName>homologene</DbName>
	<DbName>medgen</DbName>
	<DbName>mesh</DbName>
	<DbName>ncbisearch</DbName>
	<DbName>nlmcatalog</DbName>
	<DbName>omim</DbName>
	<DbName>orgtrack</DbName>
	<DbName

In [54]:
r = Entrez.efetch(db='biosample', id=zList[:10].tolist(), retmode='xml')

In [55]:
res = Entrez.parse(r, validate=False)

In [56]:
for b in res:
    pass

ValueError: The XML file does not represent a list. Please use Entrez.read instead of Entrez.parse